In [ ]:
# 필요 라이브러리 설치 및 import
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time
import matplotlib.pyplot as plt
from scipy.spatial import distance
import numpy as np

In [ ]:
# KRX 한국 주식 정보 엑셀파일 불러오기
file = pd.read_excel('/content/data_3741_20230913.xlsx')
df = pd.DataFrame(file)
code_num = df['단축코드'] # 종목 별 고유코드

In [ ]:
# 웹크롤링을 이용한 한국 주식 종목 별 카테고리 산정
category_kor = list()

for i in range(len(code_num)):
  code = code_num[i]
  specific_options = webdriver.ChromeOptions()
  specific_options.add_argument('--headless')
  specific_options.add_argument('--no-sandbox')
  driver = webdriver.Chrome(options = specific_options)
  driver.get('https://finance.naver.com/item/main.naver?code='+code)

  soup = BeautifulSoup(driver.page_source, 'html.parser')
  program = soup.find_all(class_ = 'h_sub sub_tit7')
  category = list()

  for j in program:
    j = j.text
    j = j.strip()
    category.append(j)

  if category == []:
    category_kor.append("NONE")
  else:
    category_after = category[0].split(':')
    category_after_2 = category_after[1].split('｜')
    category_after_3 = category_after_2[0].replace(" ","")
    category_kor.append(category_after_3)

category_kor = pd.DataFrame(category_kor)

In [ ]:
# 산출한 한국 주식 종목들의 카테고리를 엑셀로 저장
category_kor.to_excel("category_kor.xlsx")

In [ ]:
# 미국 주식 정보 엑셀파일 불러오기
file_2 = pd.read_excel('/content/NASDAQ_FC_STK_IEM_IFO.xlsx')
df_2 = pd.DataFrame(file_2)
code_num_2 = df_2['tck_iem_cd'] # 종목 별 고유코드

In [ ]:
# 웹크롤링을 이용한 미국 주식 종목 별 카테고리 산정
category_us = list()

for i in range(len(code_num_2)):
  code = code_num_2[i].replace(" ","")
  code = code.upper()
  specific_options = webdriver.ChromeOptions()
  specific_options.add_argument('--headless')
  specific_options.add_argument('--no-sandbox')
  driver = webdriver.Chrome(options = specific_options)
  driver.get('https://m.stock.naver.com/worldstock/stock/'+code+'.O/total')
  url = 'https://m.stock.naver.com/worldstock/stock/'+code+'.O/total'
  time.sleep(1)

  soup = BeautifulSoup(driver.page_source, 'html.parser')
  program = soup.find_all(class_ = 'StockInfo_value__3l7zZ')
  category = list()

  for i in program:
    i = i.text
    i = i.strip()
    category.append(i)

  if category == []:
    category_us.append("NONE")
  else:
    category = category[7]
    category_us.append(category)

category_us = pd.DataFrame(category_us)
category_us.to_excel("category_us.xlsx") # 산출한 미국 주식 종목들의 카테고리를 엑셀로 저장

In [ ]:
# 한국 주식 종목들의 카테고리 불러오기
file_3 = pd.read_excel('/content/category_kor.xlsx')
df_3 = pd.DataFrame(file_3)
code_num_3 = df_3[0]

# 미국 주식 종목들의 카테고리 불러오기
file_4 = pd.read_excel('/content/category_us.xlsx')
df_4 = pd.DataFrame(file_4)
code_num_4 = df_4[0]

# 한국 주식 종목 중복 제거
category_kor_edit = list()
for i in code_num_3:
  if i not in category_kor_edit:
    category_kor_edit.append(i)

# 미국 주식 종목 중복 제거
category_us_edit = list()
for j in code_num_4:
  if j not in category_us_edit:
    category_us_edit.append(j)

In [ ]:
# 한국, 미국 주식 종목들의 업종 엑셀로 저장 (이후 분석에 사용할 업종 15개 채택)
pd.DataFrame(category_kor_edit).to_excel("category_kor_edit.xlsx")
pd.DataFrame(category_us_edit).to_excel("category_us_edit.xlsx")

In [ ]:
# 선택한 15개의 업종에 해당하는 한국 주식 종목 추출
file_5 = pd.read_excel('/content/국내주식.xlsx') # data_3741_20230913.xlsx 파일과 웹크롤링을 통해 얻은 종목정보(category_kor_edit.xlsx)를 엑셀을 통해 연결한 파일 ([업종] 칼럼을 추가했음.)
df_5 = pd.DataFrame(file_5)
df_kor = df_5[['단축코드', '업종', '한글 종목명']]

condition_1 = ((df_5['업종'] == '건설') | (df_5['업종'] == '담배') | (df_5['업종'] == '방송과엔터테인먼트') |
               (df_5['업종'] == '백화점과일반상점') | (df_5['업종'] == '사무용전자제품') | (df_5['업종'] == '손해보험') |
               (df_5['업종'] == '식품과기본식료품소매') | (df_5['업종'] == '우주항공과국방') | (df_5['업종'] == '은행') |
               (df_5['업종'] == '자동차') | (df_5['업종'] == '전자장비와기기') | (df_5['업종'] == '제약') |
               (df_5['업종'] == '철강') | (df_5['업종'] == '컴퓨터와주변기기') | (df_5['업종'] == '항공사'))

df_kor = df_kor.loc[condition_1]

In [ ]:
# 한국 종목 별 시가총액 웹크롤링
df_kor_code = list(df_kor['단축코드'])
category_kor_price = list()

for i in range(len(df_kor_code)):
  code = df_kor_code[i]
  specific_options = webdriver.ChromeOptions()
  specific_options.add_argument('--headless')
  specific_options.add_argument('--no-sandbox')
  driver = webdriver.Chrome(options = specific_options)
  driver.get('https://finance.naver.com/item/main.naver?code='+code)

  soup = BeautifulSoup(driver.page_source, 'html.parser')
  program = soup.find_all(class_ = 'strong')
  category = list()

  for j in program:
    j = j.text
    j = j.strip()
    category.append(j)

  price = category[0]
  price_2 = price.replace(" ", "")
  price_3 = price_2.replace("\n", "")
  price_4 = price_3.replace("\t", "")
  price_5 = price_4.replace("시가총액", "")
  price_6 = price_5.replace("원", "")
  price_7 = price_6.replace(",", "")

  a = price_7.find("조")
  b = price_7.find("억")
  c = price_7.find("만")

  if a != -1:
    if b != -1:
      if c != -1:
        price_total = int(price_7[:a]) * 1000000000000 + int(price_7[a+1:b]) * 100000000 + int(price_7[b+1:c]) * 10000
      else:
        price_total = int(price_7[:a]) * 1000000000000 + int(price_7[a+1:b]) * 100000000
    else:
      if c != -1:
        price_total = int(price_7[:a]) * 1000000000000 + int(price_7[a+1:c]) * 10000
      else:
        price_total = int(price_7[:a]) * 1000000000000
  else:
    if b != -1:
      if c != -1:
        price_total = int(price_7[:b]) * 100000000 + int(price_7[b+1:c]) * 10000
      else:
        price_total = int(price_7[:b]) * 100000000
    else:
      if c != -1:
        price_total = int(price_7[:c]) * 10000
      else:
        price_total = 0

  category_kor_price.append(price_total)

In [ ]:
# 데이터프레임에 시가총액 추가 및 정렬
df_kor['시가총액'] = category_kor_price
df_kor = df_kor.sort_values(by = ['업종', '시가총액'], ascending = [True, False])

df_kor.to_excel("국내업종및시가총액.xlsx") # 한국 주식 종목 엑셀로 저장

In [ ]:
# 선택한 15개의 업종에 속해있는 미국 주식 추출
file_6 = pd.read_excel('/content/해외주식.xlsx') # content/NASDAQ_FC_STK_IEM_IFO.xlsx 파일과 웹크롤링을 통해 얻은 종목정보(category_us_edit.xlsx)를 엑셀을 통해 연결한 파일 ([업종] 칼럼을 추가했음.)
df_6 = pd.DataFrame(file_6)
df_us = df_6[['tck_iem_cd', '업종', 'fc_sec_eng_nm']]

condition_2 = ((df_6['업종'] == '건설 및 엔지니어링') | (df_6['업종'] == '담배') | (df_6['업종'] == '방송') |
               (df_6['업종'] == '백화점') | (df_6['업종'] == '사무기기') | (df_6['업종'] == '손해보험') |
               (df_6['업종'] == '식품 소매 및 유통') | (df_6['업종'] == '항공우주 및 방위') | (df_6['업종'] == '은행') |
               (df_6['업종'] == '자동차 및 트럭 제조') | (df_6['업종'] == '전자 장비 및 부품') | (df_6['업종'] == '제약') |
               (df_6['업종'] == '철 및 강철') | (df_6['업종'] == '컴퓨터 하드웨어') | (df_6['업종'] == '항공사'))

df_us = df_us.loc[condition_2]

In [ ]:
# 미국 종목 별 시가총액 웹크롤링
df_us_code = list(df_us['tck_iem_cd'])
category_us_price = list()

for i in range(len(df_us_code)):
  code = df_us_code[i].replace(" ","")
  code = code.upper()
  specific_options = webdriver.ChromeOptions()
  specific_options.add_argument('--headless')
  specific_options.add_argument('--no-sandbox')
  driver = webdriver.Chrome(options = specific_options)
  driver.get('https://m.stock.naver.com/worldstock/stock/'+code+'.O/total')
  url = 'https://m.stock.naver.com/worldstock/stock/'+code+'.O/total'
  time.sleep(1)

  soup = BeautifulSoup(driver.page_source, 'html.parser')
  program = soup.find_all(class_ = 'StockInfo_valueDesc__1uIaD')
  category = list()

  for i in program:
    i = i.text
    i = i.strip()
    category.append(i)

  if category != []:
    price = category[0]
    price_2 = price.replace(" ", "")
    price_3 = price_2.replace("\n", "")
    price_4 = price_3.replace("\t", "")
    price_5 = price_4.replace("시가총액", "")
    price_6 = price_5.replace("원", "")
    price_7 = price_6.replace(",", "")
    price_8 = price_7.replace(".", "")

    a = price_8.find("조")
    b = price_8.find("억")
    c = price_8.find("만")
  else:
    price_8 = '0'
    a = -1
    b = -1
    c = -1

  if a != -1:
    if b != -1:
      if c != -1:
        price_total = int(price_8[:a]) * 1000000000000 + int(price_8[a+1:b]) * 100000000 + int(price_8[b+1:c]) * 10000
      else:
        price_total = int(price_8[:a]) * 1000000000000 + int(price_8[a+1:b]) * 100000000
    else:
      if c != -1:
        price_total = int(price_8[:a]) * 1000000000000 + int(price_8[a+1:c]) * 10000
      else:
        price_total = int(price_8[:a]) * 1000000000000
  else:
    if b != -1:
      if c != -1:
        price_total = int(price_8[:b]) * 100000000 + int(price_8[b+1:c]) * 10000
      else:
        price_total = int(price_8[:b]) * 100000000
    else:
      if c != -1:
        price_total = int(price_8[:c]) * 10000
      else:
        price_total = 0

  category_us_price.append(price_total)

In [ ]:
# 데이터프레임에 시가총액 추가
df_us['시가총액'] = category_us_price
df_us = df_us.sort_values(by = ['업종', '시가총액'], ascending = [True, False])
df_us.to_excel("해외업종및시가총액.xlsx") # 미국 주식 종목 엑셀로 저장

In [ ]:
# 국내 주식의 일자별 주가 정보 불러오기 및 통합
data_before = pd.read_excel('/content/20230103.xlsx') # KRX 제공 전종목 시세 (2023년 1월 3일 기준)
data_after = pd.DataFrame(data_before)
data_after['일자'] = '20230103'

date_list = ['20230104','20230105','20230106','20230109','20230110','20230111','20230112','20230113','20230116','20230117','20230118',
             '20230119','20230120','20230125','20230126','20230127','20230130','20230131','20230201','20230202','20230203','20230206',
             '20230207','20230208','20230209','20230210','20230213','20230214','20230215','20230216','20230217','20230220','20230221',
             '20230222','20230223','20230224','20230227','20230228','20230302','20230303','20230306','20230307','20230308','20230309',
             '20230310','20230313','20230314','20230315','20230316','20230317','20230320','20230321','20230322','20230323','20230324',
             '20230327','20230328','20230329','20230330','20230331','20230403','20230404','20230405','20230406','20230407','20230410',
             '20230411','20230412','20230413','20230414','20230417','20230418','20230419','20230420','20230421','20230424','20230425',
             '20230426','20230427','20230428','20230502','20230503','20230504','20230508','20230509','20230510','20230511','20230512',
             '20230515','20230516','20230517','20230518','20230519','20230522','20230523','20230524','20230525','20230526','20230530',
             '20230531','20230601','20230602','20230605','20230607','20230608','20230609','20230612','20230613','20230614','20230615',
             '20230616','20230619','20230620','20230621','20230622','20230623','20230626','20230627','20230628','20230629','20230630',
             '20230703','20230704','20230705','20230706','20230707','20230710','20230711','20230712','20230713','20230714','20230717',
             '20230718','20230719','20230720','20230721','20230724','20230725','20230726','20230727','20230728','20230731','20230801',
             '20230802','20230803','20230804','20230807','20230808','20230809','20230810','20230811','20230814','20230816','20230817',
             '20230818','20230821','20230822','20230823','20230824','20230825','20230828','20230829','20230830','20230831']

data_list_len = len(date_list)

# KRX 제공 전종목 시세 데이터 (2023년 1월 3일 ~ 2023년 8월 31일) [일자] 칼럼 생성 및 모두 통합
for i in range(data_list_len):
  date = date_list[i]
  data_before_2 = pd.read_excel('/content/'+date+'.xlsx')
  data_after_2 = pd.DataFrame(data_before_2)
  data_after_2['일자'] = date
  data_after = pd.concat([data_after, data_after_2])

data_after.to_excel('data_integrated.xlsx')

In [ ]:
date_list_kor = ['20230104','20230105','20230106','20230109','20230110','20230111','20230112','20230113','20230116','20230117','20230118',
             '20230119','20230120','20230125','20230126','20230127','20230130','20230131','20230201','20230202','20230203','20230206',
             '20230207','20230208','20230209','20230210','20230213','20230214','20230215','20230216','20230217','20230220','20230221',
             '20230222','20230223','20230224','20230227','20230228','20230302','20230303','20230306','20230307','20230308','20230309',
             '20230310','20230313','20230314','20230315','20230316','20230317','20230320','20230321','20230322','20230323','20230324',
             '20230327','20230328','20230329','20230330','20230331','20230403','20230404','20230405','20230406','20230407','20230410',
             '20230411','20230412','20230413','20230414','20230417','20230418','20230419','20230420','20230421','20230424','20230425',
             '20230426','20230427','20230428','20230502','20230503','20230504','20230508','20230509','20230510','20230511','20230512',
             '20230515','20230516','20230517','20230518','20230519','20230522','20230523','20230524','20230525','20230526','20230530',
             '20230531','20230601','20230602','20230605','20230607','20230608','20230609','20230612','20230613','20230614','20230615',
             '20230616','20230619','20230620','20230621','20230622','20230623','20230626','20230627','20230628','20230629','20230630',
             '20230703','20230704','20230705','20230706','20230707','20230710','20230711','20230712','20230713','20230714','20230717',
             '20230718','20230719','20230720','20230721','20230724','20230725','20230726','20230727','20230728','20230731','20230801',
             '20230802','20230803','20230804','20230807','20230808','20230809','20230810','20230811','20230814','20230816','20230817',
             '20230818','20230821','20230822','20230823','20230824','20230825','20230828','20230829','20230830','20230831']

date_list_us = ['20230103','20230104','20230105','20230108','20230109','20230110','20230111','20230112','20230115','20230116','20230117',
             '20230118','20230119','20230124','20230125','20230126','20230129','20230130','20230131','20230201','20230202','20230205',
             '20230206','20230207','20230208','20230209','20230212','20230213','20230214','20230215','20230216','20230219','20230220',
             '20230221','20230222','20230223','20230226','20230227','20230301','20230302','20230305','20230306','20230307','20230308',
             '20230309','20230312','20230313','20230314','20230315','20230316','20230319','20230320','20230321','20230322','20230323',
             '20230326','20230327','20230328','20230329','20230330','20230402','20230403','20230404','20230405','20230406','20230409',
             '20230410','20230411','20230412','20230413','20230416','20230417','20230418','20230419','20230420','20230423','20230424',
             '20230425','20230426','20230427','20230501','20230502','20230503','20230507','20230508','20230509','20230510','20230511',
             '20230514','20230515','20230516','20230517','20230518','20230521','20230522','20230523','20230524','20230525','20230529',
             '20230530','20230531','20230601','20230604','20230606','20230607','20230608','20230611','20230612','20230613','20230614',
             '20230615','20230618','20230619','20230620','20230621','20230622','20230625','20230626','20230627','20230628','20230629',
             '20230702','20230703','20230704','20230705','20230706','20230709','20230710','20230711','20230712','20230713','20230716',
             '20230717','20230718','20230719','20230720','20230723','20230724','20230725','20230726','20230727','20230730','20230731',
             '20230801','20230802','20230803','20230806','20230807','20230808','20230809','20230810','20230813','20230815','20230816',
             '20230817','20230820','20230821','20230822','20230823','20230824','20230827','20230828','20230829','20230830']


In [ ]:
# 일자까지 통합한 한국 및 미국 주식 정보 엑셀파일 불러오기
file_kor = pd.read_excel('/content/국내종목정리.xlsx',dtype = "str") # 해외업종및시가총액.xlsx 파일에 [20230103] ~ [20230831] 칼럼을 엑셀을 통해 추가, 이때 각 셀의 값은 0으로 지정
file_us = pd.read_excel('/content/해외종목정리.xlsx' ,dtype = "str") # 국내업종및시가총액.xlsx 파일에 [20230103] ~ [20230831] 칼럼을 엑셀을 통해 추가, 이때 각 셀의 값은 0으로 지정
file_kor_p = pd.read_excel('/content/국내주식통합.xlsx' ,dtype = "str") # data_integrated.xlsx 파일명을 국내주식통합으로 변경 후 사용
file_us_p = pd.read_excel('/content/해외주식통합.xlsx' ,dtype = "str") # NASDAQ_DT_FC_STK_QUT.xlsx 파일명을 해외주식통합으로 변경 후 사용

data_kor = pd.DataFrame(file_kor)
data_us = pd.DataFrame(file_us)
data_kor_p = pd.DataFrame(file_kor_p)
data_us_p = pd.DataFrame(file_us_p)

In [ ]:
# 한국 주식 종목 별 등락 전처리
for i in date_list_kor:
  for j in list(data_kor['단축코드']):
    condition = (data_kor_p['일자'] == i)
    condition_2 = (data_kor_p['종목코드'] == j)
    condition_3 = (data_kor['단축코드'] == j)
    condition_total = (condition & condition_2)
    sep = data_kor_p.loc[condition_total]
    dif = 0
    dif = int(sep['종가'].iloc[0]) - int(sep['시가'].iloc[0])
    data_kor.loc[condition_3]
    if dif > 0:
      data_kor.loc[condition_3, int(i)]= 1
    elif dif == 0:
      data_kor.loc[condition_3, int(i)] = 0
    elif dif < 0:
      data_kor.loc[condition_3, int(i)] = -1

In [ ]:
# 전처리한 데이터 엑셀로 저장
data_kor.to_excel("국내최종본.xlsx")

In [ ]:
# 미국 주식 종목 별 등락 전처리
for i in date_list_us:
  for j in list(data_us['단축코드']):
    condition = (data_us_p['trd_dt'] == i)
    condition_2 = (data_us_p['tck_iem_cd'] == j.replace(" ",""))
    condition_3 = (data_us['단축코드'] == j)
    condition_total = (condition & condition_2)
    sep = data_us_p.loc[condition_total]
    if sep.empty == False:
      dif = float(sep['gts_iem_end_pr'].iloc[0]) - float(sep['gts_iem_ong_pr'].iloc[0])
      data_us.loc[condition_3]
      if dif > 0:
        data_us.loc[condition_3, int(i)] = 1
        print(1)
      elif dif == 0:
        data_us.loc[condition_3, int(i)] = 0
        print(2)
      elif dif < 0:
        data_us.loc[condition_3, int(i)] = -1
        print(3)
    else:
      data_us.loc[condition_3, int(i)] = 'NONE'

In [ ]:
# 전처리한 데이터 엑셀로 저장
data_us.to_excel("해외최종본.xlsx")

In [ ]:
# 이후 엑셀을 통해 업종에 대해 부분합을 진행하여 일자 별 평균 등락 데이터를 구함. (국내최종본.xlsx과 해외최종본.xlsx에 대하여 모두 진행, 총 row의 개수 15개.)
# 하루 일자 차이를 두고 비교하기 위해 국내최종본에서 [20230103] 칼럼 삭제
# 이후 국내최종본.xlsx과 해외최종본.xlsx에서 특정 일자에 하나라도 빈 셀이나 NONE 값이 있다면 해당 칼럼 각 파일에서 모두 삭제
# ex) 해외최종본.xlsx의 [20230108] 칼럼과 국내최종본.xlsx의 [20230109] 칼럼 삭제
# 모두 진행 후 등락정규화국내.xlsx과 등락정규화해외.xlsx로 파일명 변경

In [ ]:
# 업종 별 등락 시계열 비교 그래프
file_n_kor = pd.read_excel('/content/등락정규화국내.xlsx')
file_n_us = pd.read_excel('/content/등락정규화해외.xlsx')
data_n_kor = pd.DataFrame(file_n_kor)
data_n_us = pd.DataFrame(file_n_us)

category_list_kor = ['건설', '담배', '방송과엔터테인먼트', '백화점과일반상점', '사무용전자제품', '손해보험', '식품과기본식료품소매',
                     '우주항공과국방', '은행', '자동차', '전자장비와기기', '제약', '철강', '컴퓨터와주변기기', '항공사']
category_list_us = ['건설 및 엔지니어링', '담배', '방송', '백화점', '사무기기', '손해보험', '식품 소매 및 유통', '항공우주 및 방위', '은행',
                    '자동차 및 트럭 제조', '전자 장비 및 부품', '제약', '철 및 강철', '컴퓨터 하드웨어', '항공사']

time_series_simularity = list()

fig, axes = plt.subplots(5,3)
fig.set_size_inches((20, 15))

for i in range(len(category_list_kor)):
  condition = data_n_kor['업종'] == category_list_kor[i]
  condition_2 = data_n_us['업종'] == category_list_us[i]

  data_n_kor_2 = data_n_kor.loc[condition]
  data_n_kor_3 = data_n_kor_2.iloc[:,2:]

  data_n_us_2 = data_n_us.loc[condition_2]
  data_n_us_3 = data_n_us_2.iloc[:,2:]

  array_value_kr = np.array(data_n_kor_3.iloc[0])
  array_value_us = np.array(data_n_us_3.iloc[0])

  distance = np.linalg.norm(array_value_kr-array_value_us)
  time_series_simularity.append(distance)

  axes[i//3, i%3].plot(array_value_kr)
  axes[i//3, i%3].plot(array_value_us)

  print(time_series_simularity) # 유사도 출력